In [ ]:
import re
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from transformers import BertTokenizer,BertForSequenceClassification,Trainer,TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder
import torch
from sklearn.model_selection import train_test_split
from google.colab import drive
!pip install datasets
!pip install evaluate
from datasets import Dataset
import evaluate
from scipy.special import expit
#

drive.mount('/content/drive')
##
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
#####PREPROCESSING####
#read file lines
'''
with open('/content/drive/MyDrive/Data/BGL.log','r') as file:
  lines = file.readlines()#perno tis grammes tou arxeiou
  lines = [line.strip() for line in lines]#list comprehension gia an vgalo ta newlines
###
log_templates = pd.read_csv('/content/drive/MyDrive/Data/BGL_templates.csv')

##
part = r"(?P<Label>.+)\s(?P<Timestamp>\d{10})\s(?P<Date>\d{4}\.\d{2}\.\d{2})\s(?P<Node>\S+)\s(?P<Time>\S+)\s(?P<NodeRepeat>\S+)\s(?P<Type>\S+)\s(?P<Component>\S+)\s(?P<Level>\S+)"
regex = rf"{part}(\s(?P<Content>.+))?"
#to f gia antikatastasi {variable}
#
#list comprehension - vazo me ayth th texniki se mia lista to dictionary pou prokyptei apo ta matches kai tis antistixiseis tous sta groups
lines = [(re.match(regex,line)).groupdict() for line in lines]

print(lines[0:2])
#To dataframe
logs = pd.DataFrame.from_dict(lines)
display(HTML((logs.sample(1)).to_html()))
####
'''
logs = pd.read_csv('/content/drive/MyDrive/Data/BGL_2k.log_structured.csv')

###CONVERT TO DATETIME####
logs['Time'] = pd.to_datetime(logs['Time'],format="%Y-%m-%d-%H.%M.%S.%f")
#drop useless columns
logs.drop(columns=['Timestamp','Date','NodeRepeat','Type','Component','Level'],inplace=True)
##NO NULL VALUES
print('Class Distribution: ',logs['Label'].value_counts())
print('Nulls of Content',logs['Content'].isnull().sum())
print('Nulls of Time: ',logs['Time'].isnull().sum())
#
print('Nulls of Node: ',logs['Node'].isnull().sum())
#
logs['Node'] = logs['Node'].fillna('NULL')

#
print('Nulls of Node: ',logs['Node'].isnull().sum())

#
print('Lines where Node is NULL',logs['Node'].value_counts().get('NULL',0))
print('Lines where Node is UNKNOWN_LOCATION',logs['Node'].value_counts().get('UNKNOWN_LOCATION',0))
##MPORO NA RIKSO KAI TA LINES ME NULL NODES H UNKNOWN LOCATION EFOSON STO UNKNOWN LOCATION MPOREI
#NA ANAFERETAI SE DIAFORETIKA NODES EN TELEI ARA DEN SXETIZONTAI NO CONTEXT H KAI OXI
#print('Random null content row: ',(logs[logs['Content'].isnull()]).sample(1))
##
#display(HTML(((logs[logs['Content'].isnull()]).sample(1)).to_html()))
input('WAIT')
###Fill None Content with empty string or drop them(de tha to proteina-alla meionei ton ogko ton dedomenon)##
logs['Content'] = logs['Content'].fillna('')
##Drop unknown nodes or null nodes###
#logs = logs[(logs['Node'] != 'NULL') & (logs['Node'] != 'UNKNOWN_LOCATION')]

###
#####
##
##CLASS/LABELS ENCODING######
##
'''
encoder = LabelEncoder()
logs['Label'] = encoder.fit_transform(logs['Label'])
'''
#############################

logs = logs.sort_values(by=['Node','Time']).groupby('Node',dropna=False)#sortare vash nodeid kai meta xronou oste kathe group na einai xronika sortarismeno kai meta groupare ta vash nodeID
#
ranges = logs.agg(MinTime=('Time', 'min'),MaxTime=('Time', 'max'))
ranges['TimeRangeDays'] = (ranges['MaxTime'] - ranges['MinTime']).dt.days
print('Mean duration of each node: ',ranges['TimeRangeDays'].mean())
#
#input('WAIT')
##GROUPING IN FIXED TIME WINDOWS WITH NODEID AND WINDOW SIZE##
#windows = pd.DataFrame(columns=['Node','Window Start','text','labels'])#ena label column alla ekkremei pos tha ypologistei
windows = []
#########
def create_time_windows(group):
  global windows

  #vale sto windows dataframe to id tou node os mia stili,allh stili to start tou window, sthlh text combined ola ta content tou xronikou group/>se deyterh fash to label
  #enose ola ta content tou time window se mia log paragraph
  #
  ##
  time_windows = group.groupby(pd.Grouper(key='Time',freq="1h"))#ftiakse ta parathyra aytou tou node
  #ena groupby object einai ena tuple ID-Data ego xrhsimopoio ta data
  for start,subgroup in time_windows:#ena grouby object epistrefei ena tuple tou typou (id,data) xrhsimopoio ta data kai to id paei axrisimopoihto
    #joinare ola ta strings tou subgroup kai valta sto neo dataframe
    text = (' '.join(subgroup['Content']))#strip kostizei
    node_name = group.name
    #window_start = subgroup['Time'].min()
    window_start = start
    #
    labels = 0
    if len(subgroup['Label']) == 0:
      labels = 0
    else:
      anomaly_percentage = len(subgroup[subgroup['Label'] != '-']) / len(subgroup['Label'])
      labels = 1 if anomaly_percentage > 0.05 else 0

    data = {'Node': node_name, 'Window Start': window_start,'text': text,'labels':labels}  # ta [] ta thelei alios peta error kati gia scalar values
    #windows = pd.concat([windows, data]) -->>OXIIII PIANEI PARA POLLH MNHMH MH APODOTIKO
    #vale os window start h to key tou subgroup h to min ton datetimes tou group
    #vale os nodeid to group.name
    windows.append(data)
  #
  ##
  return
######APPLY THE FUNCTION TO THE LOGS#######
######TO PARATHYRO PREPEI NA PAREI KAI KAPOIO LABEL MPORO PX ME DIAFOROUS TROPOUS
####EPEIDH EINAI POLLA TA BENIGN GIA KALYTERO BALANCE THA MPOROUSA PX AKOMA KAI
###ENA ALLO LABEL NA YPARXEI STO GROUP PERA TOU BENIGN NA TO THEORISO ANOMALY
###H ME KAPOIO THRESHOLD TON MH BENIGN KAI PANO NA THEOREITAI ANOMALY
logs.apply(create_time_windows)
###
windows = pd.DataFrame(windows)
windows.info()
print('Distribution of classes',windows['labels'].value_counts())
input('WAIT')
#######GIA PERETERO EKSIKONOMISI PERA APO UNKNOWN/NULL EDO THA MPOROUSA NA RIKSO KAPOIA PARATHYRA ALLA TA PARATHYRA OXI TA LOG LINES!##
#########
##Templates-Log keys###
'''
event_dictionary = dict(zip(log_templates['EventId'],log_templates['EventTemplate']))
print('Dictionary of template Ids : template Content : ',event_dictionary)
'''
#########
######TRAIN-TEST-VALIDATION SPLIT######

windows_train,windows_test = train_test_split(windows,test_size=0.1,random_state=42,shuffle=True)
##
windows_train,windows_validation = train_test_split(windows_train,test_size=0.1111,random_state=42,shuffle=True)
######Dataframe to HuggingFace Dataset#####

windows_train = Dataset.from_pandas(windows_train)
windows_test = Dataset.from_pandas(windows_test)
windows_validation = Dataset.from_pandas(windows_validation)
###########
###TOKENIZER && TOKENIZING####
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased") # tokenizer

def tokenize_logs(entry):
  tokens = tokenizer(entry['text'],padding='max_length',truncation=True)
  #tokens['labels'] = entry['Label']
  return tokens
###TOKENIZING SUB DATASETS######
windows_train = windows_train.map(tokenize_logs,batched=True)
windows_test = windows_test.map(tokenize_logs,batched=True)
windows_validation = windows_validation.map(tokenize_logs,batched=True)

##METRICS####
accuracy = evaluate.load("accuracy")
other_metrics = evaluate.combine(["precision","recall","f1"])
roc_auc = evaluate.load("roc_auc")
#combine all metrics to a dictionary
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    prediction_scores = expit(predictions)#expit(logits)
    roc_auc_score = roc_auc.compute(references=labels,prediction_scores=prediction_scores[:,1])#y_true,y_score
    predictions = np.argmax(predictions, axis=-1)
    other_metrics_scores = other_metrics.compute(predictions=predictions,references=labels,average=None)#kathe klash
    accuracy_score = accuracy.compute(predictions=predictions,references=labels)["accuracy"]#epistrefei dict kai perno thn timh tou accuracy:
    return {"accuracy":accuracy_score,"precision_class_0":other_metrics_scores["precision"][0],"precision_class_1":other_metrics_scores["precision"][1],"recall_class_0":other_metrics_scores["recall"][0],"recall_class_1":other_metrics_scores["recall"][1],"f1_class_0":other_metrics_scores["f1"][0],"f1_class_1":other_metrics_scores["f1"][1],"roc_auc":roc_auc_score["roc_auc"]}

###METRICS#####

###MODEL###
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels=2).to(device)#modelo me ena extra layer gia to classification
###TRAINING ARGS#####
training_arguments = TrainingArguments(
    output_dir = "/content/drive/MyDrive/Run",#pou tha apothikeytei to modelo kai ta checkpoints. gia persistency to vazo sto /content/MyDrive/...
    overwrite_output_dir=True,  # MONO GIA THN PROTH FORA META PREPEI FALSE
    eval_strategy = "steps",#to evaluation ginetai sto telos kathe synolou steps
    learning_rate=2e-5,#rythmos mathisis genika na einai mikros xoris akrotites. mikrh timh-> kalyterh genikeysh
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,#genika de thelei oute poly liga alla oute polla.poses fores to modelo tha iterarei to training dataset-koinos se posa epochs tha ginei to training
    weight_decay=0.01,#meiosh tou overfitting
    resume_from_checkpoint=True,
    save_steps=500,  # Save model every 500 steps
    save_total_limit=2  # Keep only the last 2 checkpoints to save disk space
    #kapoia akoma orismata + fakelos
)
####TRAINER###
trainer = Trainer(
    model=model,#modelo pou tha kanei to classification,
    args=training_arguments,#ta orismata tis ekpaideyshs apo pano
    #tokenizer=tokenizer#o tokenizer, an exo kanei hdh preprocessing-tokenizing LOGIKA den to xreiazomai
    train_dataset=windows_train,#to synolo ekpaideyshs gia to fine-tuning
    eval_dataset=windows_validation,#to synolo elegxou-aksiologisis gia to fine tuning-xoris ayto de tha aksiologithei h apodosi tou modelou mono tha ekpaideytei
    #tokenizer=tokenizer
    compute_metrics=compute_metrics#exei os apotelesma sto training tou modelou ektos apo to training/validation loss na emfanizontai kai oi times ton metrikon pou orisa sth synartisi kai epistrefontai apo ayth
    #
    )

####TRAIN && EVALUATE
trainer.train()# to orisma RESUME benei AFOU ginei interrupt!-to resume-from-checkpoint=directory
results = trainer.evaluate(eval_dataset=windows_test) #ti apodosi fernei to modelo sto test dataset-orisma to test dataset
print(results)




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cuda
Class Distribution:  Label
-           1857
KERNDTLB      60
KERNSTOR      30
APPSEV        17
KERNMNTF      11
KERNTERM       7
KERNREC        5
APPRES         4
APPREAD        3
APPTO          2
KERNRTSP       2
APPCHILD       1
APPOUT         1
Name: count, dtype: int64
Nulls of Content 0
Nulls of Time:  0
Nulls of Node:  35
Nulls of Node:  0
Lines where Node is NULL 35
Lines where Node is UNKNOWN_LOCATION 10
WAIT
Mean duration of each node:  1.6141732283464567


<ipython-input-3-3f606724ae9d>:135: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  logs.apply(create_time_windows)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71252 entries, 0 to 71251
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Node          71252 non-null  object        
 1   Window Start  71252 non-null  datetime64[ns]
 2   text          71252 non-null  object        
 3   labels        71252 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 2.2+ MB
Distribution of classes labels
0    71162
1       90
Name: count, dtype: int64
WAIT


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,Accuracy,Precision Class 0,Precision Class 1,Recall Class 0,Recall Class 1,F1 Class 0,F1 Class 1,Roc Auc
500,0.018100,0.003144,0.998456,0.998456,0.000000,1.000000,0.000000,0.999227,0.000000,0.999949
1000,0.002300,0.002147,0.999439,0.999578,0.888889,0.999859,0.727273,0.999719,0.800000,0.999847
1500,0.001900,0.001082,0.999719,1.000000,0.846154,0.999719,1.000000,0.999859,0.916667,0.999962
2000,0.000100,0.000669,0.999860,1.000000,0.916667,0.999859,1.000000,0.999930,0.956522,0.999962
2500,0.000900,0.000672,0.999860,1.000000,0.916667,0.999859,1.000000,0.999930,0.956522,1.000000
3000,0.000600,0.000789,0.999860,1.000000,0.916667,0.999859,1.000000,0.999930,0.956522,0.999962
3500,0.001000,0.000331,0.999860,1.000000,0.916667,0.999859,1.000000,0.999930,0.956522,1.000000
4000,0.000100,0.000699,0.999860,1.000000,0.916667,0.999859,1.000000,0.999930,0.956522,0.999962


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
